In [13]:
import pandas as pd
import numpy as np

In [16]:
#import data in the form of csv file
trans_data = pd.read_csv('transitions_data.txt',header=None)
#rd_data = pd.read_csv('rewards.csv',header=None)
rwd_data = pd.read_csv('rewards.txt',header=None)
#convert the pandas file into numpy array or matrix
trans_data = trans_data.to_numpy()
#t_data = pd.read_csv('transitions_data.csv',header=None)
rwd_data = rwd_data.to_numpy()

In [20]:
gamma = 0.9

In [33]:
transitions = {}
len_data = trans_data.shape[0]
td = trans_data
for i in range(1,len_data):
  if (td[i][0] in transitions):
    if td[i][1] in transitions[td[i][0]]:
      transitions[td[i][0]][td[i][1]].append((float(td[i][3]),td[i][2]))
    else:
      transitions[td[i][0]][td[i][1]] = [(float(td[i][3]),td[i][2])]
  else:
    transitions[td[i][0]] = {td[i][1]:[(float(td[i][3]),td[i][2])]}
  print(trans_data.shape)
for i in transitions.keys():
  print(i)

(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(110, 4)
(3 0)
(3 1)
(1 0)
(

In [32]:
rewards = {}
rd = rwd_data
len_rewards = rd.shape[0]
for i in range(0,len_rewards):
  rewards[rd[i][0]] = float(rd[i][1]) if rd[i][1] != 'None' else np.nan
print(len(rewards.keys()))

12


In [25]:
rkeys = rewards.keys()
tkeys = transitions.keys()
st = ''
st2 = ''
for i in tkeys:
  st += i + ' '
for j in rkeys:
  st2 += j + ' '
print(st)
print(st2)

(3 0) (3 1) (1 0) (2 1) (1 2) (2 0) (3 2) (2 2) (0 1) (0 0) (0 2) 
(0 1) (3 0) (0 0) (1 0) (3 1) (1 1) (2 1) (1 2) (2 0) (3 2) (0 2) (2 2) 


In [26]:
class MarkovDecisionProcess:
  def __init__(self, states=[], transition={}, reward={}, gamma=0.9):
    self.states = states
    self.transition = transition
    self.reward = reward
    self.gamma = gamma

  def Rwd(self, state):
    return self.reward[state]
  
  def Trans(self, state, action):
    return self.transition[state][action]

  def action(self, state):
    return self.transition[state].keys()
     

In [27]:
Transitions = transitions
Rewards = rewards
States = transitions.keys()
print(States)

dict_keys(['(3 0)', '(3 1)', '(1 0)', '(2 1)', '(1 2)', '(2 0)', '(3 2)', '(2 2)', '(0 1)', '(0 0)', '(0 2)'])


In [28]:
mdp = MarkovDecisionProcess(states = States, transition = Transitions, reward = Rewards)
epsilon = 0.2
def val_iteration():
  states = mdp.states
  actions = mdp.action
  Trans = mdp.Trans
  Rwd = mdp.Rwd

  #initializing the policy all with 0 value
  V1 = {s: 0 for s in states}
  while True:
    V = V1.copy()
    delta = 0 

    for s in states:
      #Synchrounous Bellman update, updating the utility values
      #two times or nested list comprehension done two times
      V1[s] = Rwd(s) + gamma * max([sum([p*V[s1] for (p,s1) in Trans(s,a)]) for a in actions(s)])
      #calculating the max difference in subsequent iterations 
      delta = max(delta, abs(V1[s]-V[s]))

    if (delta < epsilon*(1-gamma)/gamma):
      return V

In [29]:
def expected_utility(a,s,V):
  Trans = mdp.Trans
  return sum([p*V[s1] for (p,s1) in Trans(s,a)])

In [30]:

def best_policy(V):
  states = mdp.states
  actions = mdp.action
  pi_policy = {}
  for s in states:
    pi_policy[s] = max(actions(s), key=lambda a: expected_utility(a,s,V))
  return pi_policy

In [31]:

V = val_iteration()
print("State-Value")
for s in V:
  print(s,' - ',V[s])
pi = best_policy(V)
print("\n Optimal policy is \n State - Action ")
for s in V:
  print(s," ",'-'," ",pi[s])

State-Value
(3 0)  -  2.6118518779566258
(3 1)  -  -10.0
(1 0)  -  4.0484603014043845
(2 1)  -  5.633133936112791
(1 2)  -  7.348949178462026
(2 0)  -  4.611938876757595
(3 2)  -  10.0
(2 2)  -  8.425245372413883
(0 1)  -  5.49111436850307
(0 0)  -  4.691426415451711
(0 2)  -  6.312795272532248

 Optimal policy is 
 State - Action 
(3 0)   -   L
(3 1)   -   EXIT
(1 0)   -   L
(2 1)   -   U
(1 2)   -   R
(2 0)   -   U
(3 2)   -   EXIT
(2 2)   -   R
(0 1)   -   U
(0 0)   -   U
(0 2)   -   R
